# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-01-23 08:06:13] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-01-23 08:06:13] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-01-23 08:06:13] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2026-01-23 08:06:16] INFO server_args.py:1724: Attention backend not specified. Use fa3 backend by default.


[2026-01-23 08:06:16] INFO server_args.py:2618: Set soft_watchdog_timeout since in CI


...

Capturing batches (bs=128 avail_mem=76.34 GB):   0%|          | 0/20 [00:00<?, ?it/s]

Capturing batches (bs=72 avail_mem=76.20 GB):  25%|██▌       | 5/20 [00:00<00:01,  9.58it/s]

Capturing batches (bs=32 avail_mem=76.18 GB):  60%|██████    | 12/20 [00:00<00:00, 17.11it/s]

Capturing batches (bs=12 avail_mem=76.16 GB):  75%|███████▌  | 15/20 [00:01<00:00, 14.74it/s]

Capturing batches (bs=1 avail_mem=76.15 GB): 100%|██████████| 20/20 [00:01<00:00, 14.76it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Ian and I live in West Perth, Perth, Western Australia. I have been training and playing on a golf course for 13 years. I enjoy keeping fit and was awarded a handicap of 2 during my first year at Perth Golf Club.
I started to play golf on my birthday in 2016. My handicap was 2 during my first year at Perth Golf Club and I started at 22. I have never played on a handicap 1 or handicap 3 course before. I was also very young when I first started playing golf and I am now 33. I have been playing golf for a
Prompt: The president of the United States is
Generated text:  trying to decide between two different policies. The first policy is to cut 20% of the federal budget. The second policy is to cut 10% of the federal budget. Which policy will result in a smaller total reduction in the federal budget?
To determine which policy will result in a smaller total reduction in the federal budget, we need to calculate the total reduction for each policy and 

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name], and I'm a [Job Title] at [Company Name]. I'm a [Number] year old, [Gender] and [Country]. I'm a [Skill] with [Number] years of experience in [Field of Work]. I'm passionate about [Why You're Interested in the Industry]. I'm always looking for ways to [What You Can Do for the Company]. I'm [What You Can Do for the Company]. I'm excited to [What You Can Do for the Company]. I'm looking forward to [What You Can Do for the Company]. I'm [What You Can Do for the Company

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, known for its iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. It is also a major center for art, culture, and politics, and is home to many world-renowned museums and theaters. Paris is a popular tourist destination, with millions of visitors each year. The city is also known for its rich history, including the influence of the French Revolution and the influence of the French language. Paris is a vibrant and dynamic city, with a diverse population and a rich cultural heritage. The city is also home to many international organizations and institutions, including the French Academy of Sciences and

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by rapid advancements in areas such as machine learning, natural language processing, and computer vision. These technologies will continue to improve, leading to more sophisticated and accurate AI systems that can perform a wide range of tasks with increasing accuracy and efficiency. Some possible future trends in AI include:

1. Increased focus on ethical considerations: As AI systems become more sophisticated, there will be a growing emphasis on ethical considerations and responsible use of AI. This will include issues such as bias, transparency, accountability, and fairness.

2. Greater integration with human decision-making: AI systems will become more integrated with human decision-making processes, allowing for more



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [insert name] and I am a [insert occupation or profession]. I bring a unique blend of knowledge, experience, and a deep respect for the natural world to the table. In my free time, I like to explore the outdoors and enjoy hiking, camping, and gardening. I enjoy creating and sharing my art, from painting to sculpture. Outside of my creative pursuits, I have a strong interest in sustainability and eco-tourism. I am always looking for new ways to bring people closer to nature and learn more about it. I am looking forward to meeting you and learning more about your interests. Good luck! [Insert name]

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, known for its iconic landmarks such as the Eiffel Tower, the Louvre Museum, and the Notre-Dame Cathedral.
The city is also famous for its history, with UNESCO li

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Name

]. I

'm a

...

 as

 predicting

 disease

 progression

 and

In [6]:
llm.shutdown()